In [29]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
pd.set_option('display.max_rows', None)

In [191]:
path1 = '/Users/USER/Dropbox/台北最愛排房表/最愛109年訂房房況表.xls'
path2 = '/Users/USER/Dropbox/台北最愛排房表/最愛110年訂房房況表.xls'

In [192]:
xls1 = pd.ExcelFile(path1)
xls2 = pd.ExcelFile(path2)

In [193]:
# read in the slide of excel
df = pd.read_excel(xls1,'109年7-12月')

In [194]:
df2 = pd.read_excel(xls2,'110年1-6月')

In [195]:
df3 = pd.read_excel(xls2,'110年7-12月')

# Data Cleaning

In [196]:
def clean(df):
    df = df.iloc[:18,1:].drop(1).T
    column_name = ['days_in_week',201,301,501,601,202,302,502,602,203,303,503,603,205,305,505,605]
    #get relevant column
    df.columns = column_name
    # get relevant rows
    df = df.iloc[1:]
    return df

In [197]:
df = clean(df)

In [198]:
df.shape

(184, 17)

In [199]:
df2 = clean(df2)

In [200]:
df2.shape

(181, 17)

In [201]:
df3 = clean(df3)

In [202]:
df2.shape

(181, 17)

In [203]:
df =df.append(df2).append(df3)

In [204]:
df.shape

(546, 17)

In [205]:
# df = df.iloc[:18,1:].drop(1).T

In [206]:
# column_name = ['days_in_week',201,
#  301,
#  501,
#  601,
#  202,
#  302,
#  502,
#  602,
#  203,
#  303,
#  503,
#  603,
#  205,
#  305,
#  505,
#  605]

In [207]:
#get relevant column
# df.columns = column_name

In [208]:
# get relevant rows
# df = df.iloc[1:]

# Data Manipulation

In [209]:
#get the boolean of current relevant day
current_date_boolean = pd.Series(df.index) >= date.today()

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [210]:
# resettig index to allow date to be accessed
df = df.reset_index()
df.rename(columns={'index':'date'}, inplace=True)

In [211]:
#filtering the date
df = df[current_date_boolean]

In [212]:
df.head()

,date,days_in_week,201,301,501,601,202,302,502,602,203,303,503,603,205,305,505,605
130,2020-11-08,日,莊玉平(($2000不含餐),恆力-HOANG印尼移工11/8~11/22($2300含餐),Elizabeth ($2300含餐),NaN,NaN,高雅玲($2500含餐),黃偉銘($2500/含餐),Ila Rosdiana($2300含餐),韓裕鯤($2500/含餐),LE THI越外配($2500含餐),符傳健($2500/含餐),施建宏($2300/含餐),林柏光11/8~11/22($2800含餐)10/27付2800訂金,NaN,石信勝($2800含餐),黃宇慶($2500/含餐)
131,2020-11-09,一,莊玉平(($2000不含餐),恆力HOANG印尼移工11/8~11/22($2300含餐),Elizabeth ($2300含餐),NaN,NaN,高雅玲($2500含餐),黃偉銘($2500/含餐),Ila Rosdiana($2300含餐),韓裕鯤($2500/含餐),LE THI越外配($2500含餐),符傳健($2500/含餐),NaN,林柏光($2800)含餐),林賢慶10/12訂(11/9~11/23含餐$2800)10/12以匯5600訂金/後5馬7...,石信勝($2800含餐),黃宇慶($2500/含餐)
132,2020-11-10,二,莊玉平(($2000不含餐),恆力HOANG印尼移工11/8~11/22($2300含餐),Elizabeth ($2300含餐),NaN,NaN,高雅玲($2500含餐),黃偉銘($2500/含餐),Ila Rosdiana($2300含餐),韓裕鯤($2500/含餐),LE THI越外配($2500含餐),符傳健($2500/含餐),NaN,林柏光($2800)含餐),林賢慶10/12訂(含餐$2800),石信勝($2800含餐),黃宇慶($2500/含餐)
133,2020-11-11,三,莊玉平(($2000不含餐),恆力HOANG印尼移工11/8~11/22($2300含餐),Elizabeth ($2300含餐),NaN,范文仲10/12訂(11/11~11/25含餐$2500)10/12已到店付2500訂金,高雅玲($2500含餐),黃偉銘($2500/含餐),Ila Rosdiana($2300含餐),韓裕鯤($2500/含餐),NaN,符傳健($2500/含餐),NaN,林柏光($2800)含餐),林賢慶10/12訂(含餐$2800),石信勝($2800含餐),黃宇慶($2500/含餐)
134,2020-11-12,四,林S-大慶泰移工11/12~11/26,恆力HOANG印尼移工11/8~11/22($2300含餐),Elizabeth ($2300含餐),林S-大慶泰移工11/12~11/26,范文仲($2500/含餐),高雅玲($2500含餐),黃偉銘($2500/含餐),Ila Rosdiana($2300含餐),韓裕鯤($2500/含餐),林S-大慶泰移工11/12~11/26,符傳健($2500/含餐),林S-大慶泰移工11/12~11/26,林柏光($2800)含餐),林賢慶10/12訂(含餐$2800),NaN,黃宇慶($2500/含餐)


In [213]:
#get rooms
rooms = [201,
 301,
 501,
 601,
 202,
 302,
 502,
 602,
 203,
 303,
 503,
 603,
 205,
 305,
 505,
 605]

# Stratify the dates that allow reservation

In [214]:
final_room_ava = pd.DataFrame()

In [215]:
#pre-allocation of number of days possible
ava_series_econ = pd.Series([0]*df.shape[0])
ava_series_stand = pd.Series([0]*df.shape[0])
ava_series_deluxe = pd.Series([0]*df.shape[0])

In [216]:
#stratify each room for 15 days continuous vacancies
for j in range(len(rooms)):
    temp_name = df.iloc[:,j+2].name
    temp_room_vacancy = df[temp_name].isna()
    for i in range(df.shape[0]-14):
        if str(temp_name)[1:] =='01':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_econ[i] += 1
        elif str(temp_name)[1:] =='05':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_deluxe[i] += 1
        else:
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_stand[i] += 1

In [217]:
final_room_ava['date'] = list(df['date'])
final_room_ava['經濟'] = ava_series_econ
final_room_ava['標準'] = ava_series_stand
final_room_ava['豪華'] = ava_series_deluxe

In [218]:
export = (final_room_ava.set_index('date').T)

In [219]:
export

date,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,...,2021-12-19,2021-12-20,2021-12-21,2021-12-22,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28
經濟,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
標準,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
豪華,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [220]:
#changing it to format friendly to excel
export.columns = pd.Series(export.columns).apply(lambda x: str(x.date().strftime('%m-%d')))

In [221]:
export.T

,經濟,標準,豪華
date,,,
11-08,0,0,0
11-09,0,0,0
11-10,0,0,0
11-11,0,0,0
11-12,0,0,0
11-13,0,0,0
11-14,0,0,0
11-15,0,0,0
11-16,0,0,0


In [222]:
export.T.to_excel('今天空房表.xls')

In [223]:
export.T.to_excel(r'/Users/USER/Dropbox/台北最愛排房表/今天空房表.xls')